## 3. Data Cleaning

Data: 12 de Junho de 2023

Esse notebook contém:


### Notas:



In [1]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from nbconf import *

import sqlite3

In [2]:
conn = sqlite3.connect(os.path.join(config.DATA_DIR, config.WEBPAGES_DATABASE))

In [3]:
cursor = conn.cursor()

In [4]:
df = pd.DataFrame(cursor.execute("SELECT * FROM metadata"))

### Data Selection

In [5]:
df.columns = [
    'url',
    'doi',
    'type',
    'author',
    'institute',
    'knowledge_area',
    'committee',
    'title_pt',
    'title_en',
    'keywords_pt',
    'keywords_en',
    'abstract_pt',
    'abstract_en',
    'publish_date'
]

In [6]:
df.head()

,url,doi,type,author,institute,knowledge_area,committee,title_pt,title_en,keywords_pt,keywords_en,abstract_pt,abstract_en,publish_date
0,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-171200,Tese de Doutorado,"Pinto, Irajá Damiani (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Corais carboníferos da Amazônia,Not available,Coral (gema orgânica) Estratigrafia - Amazônia...,Not available,O presente trabalho compreende o estudo de cor...,Not available,2016-07-07
1,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-06072016-172400,Tese de Doutorado,"Kawashita, Koji (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Metodo Rb-Sr em rochas sedimentares: aplicação...,Not available,Geocronologia Rochas sedimentares,Not available,Estudo radiométrico em sedimentos com posição ...,Not available,2016-07-07
2,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/D.44.2016.tde-06072016-172430,Dissertação de Mestrado,"Kawashita, Koji (Catálogo USP)",Instituto de Geociências,Geologia Geral,None,Um sensível espectrômetro de massa de fonte só...,Not available,Espectrometria de massas Geocronologia,Not available,Um espectrômetro de massa de fonte sólida de a...,Not available,2016-07-07
3,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-07072016-084400,Tese de Doutorado,"Szikszay, Mária (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Aspectos geoquímicos de rios da bacia do Paraná,Not available,Geoquímica - Bacia do Paraná,Not available,Os rios da bacia hidrográfica do Paraná drenam...,Not available,2016-07-07
4,https://www.teses.usp.br/teses/disponiveis/44/...,10.11606/T.44.2016.tde-07072016-085900,Tese de Doutorado,"Kanji, Milton Assis (Catálogo USP)",Instituto de Geociências,Área do Conhecimento,None,Resistência ao cisalhamento de contactos solo-...,Not available,Conservação do solo,Not available,Nos estudos referentes à estabilidade de maciç...,Not available,2016-07-07


#### Null values

In [7]:
df = df.replace(to_replace=["Not available", "não possui", "Não disponível", "No abstract"], value=np.nan)

In [8]:
nonan_indexes = df[df['title_en'].notna() & df['abstract_en'].notnull()].index

In [9]:
df = df.iloc[nonan_indexes,:]

In [10]:
df = df.reset_index(drop=True)

In [11]:
df.isna().sum()

url                  0
doi                 33
type                 0
author               0
institute            0
knowledge_area       0
committee         3527
title_pt            66
title_en             0
keywords_pt        473
keywords_en       3265
abstract_pt        122
abstract_en          0
publish_date        32
dtype: int64

In [12]:
df = df.fillna("")

In [13]:
df.isna().sum().sum()

0

#### Author

In [14]:
def remove_catalogo_usp(s):
    return s.replace("(Catálogo USP)", "")

df['author'] = df['author'].apply(remove_catalogo_usp)

### Saving

In [24]:
data_pt = df.loc[:,["title_pt", "abstract_pt", "keywords_pt"]]
metadata_pt = df.loc[:, ["url", "type", "author", "institute", "title_pt", "abstract_pt", "keywords_pt"]]

data_pt.to_csv(os.path.join(config.DATA_DIR, "data_pt.csv"), escapechar='\\', index=False)
metadata_pt.to_csv(os.path.join(config.DATA_DIR, "metadata_pt.csv"), escapechar='\\', index=False)

In [25]:
data_en = df.loc[:,["title_en", "abstract_en", "keywords_en"]]
metadata_en = df.loc[:, ["url", "type", "author", "institute", "title_en", "abstract_en", "keywords_en"]]

data_en.to_csv(os.path.join(config.DATA_DIR, "data_en.csv"), index=False)
metadata_en.to_csv(os.path.join(config.DATA_DIR, "metadata_en.csv"), index=False)

### Loading

In [ ]:
data_pt.to_dict()

In [26]:
data_pt = pd.read_csv(os.path.join(config.DATA_DIR, "data_pt.csv"))

In [249]:
data = pd.read_csv(os.path.join(config.DATA_DIR, "data.csv"), keep_default_na=False)

In [250]:
data.isna().sum()

title_en       0
abstract_en    0
keywords_en    0
dtype: int64

### Encoding

O código a seguir cria batches de descrições a partir do título, abstract e palavras-chave e solicita ao modelo que codifique as embeddings, que são apensadas a vectors ao final.

In [248]:
data = pd.read_csv(os.path.join(config.DATA_DIR, "data.csv"), keep_default_na=False)

##### Exemplo de execução

In [106]:
row = data.iloc[97601,:]
pre = (row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". ")
pre

['Proposal of a method for coalition control of virtuals resouces based on multiagents in the I4.0 context.',
 'Traditional manufacturing systems need proposals adherents to the Industry 4.0 (I4.0) context',
 "The I4.0 guidelines demonstrate the need for self-organization and plug-andproduce, demanding greater flexibility in today's applications",
 'The virtualization of different components is promising for replicating real systems in the virtual environment',
 'One area that requires attention is the different relationships and task assignments of resources so that they can be understood in the virtual world',
 'The Asset-Administration Shell (AAS) concept provides a standard for the virtualization of industry asset data, in addition to introducing the concept of capabilities engineering, meeting the requirements of an active virtual product in the production flow',
 'The cooperation of different resource abilities can be achieved with the support of control applications that perform

In [8]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device='cuda')

In [14]:
model.encode(pre).shape

(14, 768)

In [16]:
# batch of size 64
batch = [pre for _ in range(64)]
import time
s = time.time()
vectors = model.encode(batch)
e = time.time()
print(e - s, " segundos")

0.09213018417358398  segundos


In [17]:
vectors.shape

(64, 768)

In [105]:
# o texto é representado por
print(len(vectors[0]), "embeddings")

768 embeddings


#### Cálculo dos vetores

In [107]:
# encoding in batches can reduce overhead costs and speed up the process
vectors = []
batch_size = 64  # TODO: how can I speed up? Maybe reduce batch?
batch = []
for row in tqdm(data.itertuples()):
    descriptions = []
    pre = (row.title_en + ". " + row.abstract_en + " " + row.keywords_en).split(". ")
    encoding = model.encode(pre)
    vectors.append(encoding)
#     if len(batch) >= batch_size:
#         vectors.append(model.encode(batch))
#         batch = []
# if len(batch) > 0:
#     vectors.append(model.encode(batch))
#     batch = []

vectors_concat = np.concatenate(vectors)

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [226]:
np.save(os.path.join(config.DATA_PATH,"vectors_experiment_3.npy"), vectors_concat, allow_pickle=False) 

### Testes - Para ter certeza de que os vetores fazem sentido

In [103]:
vectors = np.load(os.path.join(config.DATA_DIR,"vectors.npy"))

In [73]:
from sklearn.metrics.pairwise import cosine_similarity

> **Pergunta**: Como funciona cosine_similarity?

In [238]:
import time
time.time()

1686621456.7033858

In [36]:
query_vector = model.encode("ddos attack")

In [94]:
cosine_similarity(query_vector, vectors)

array([0.30398664, 0.16937862, 0.20659798, ..., 0.04241941, 0.0696875 ,
       0.11439453], dtype=float32)

In [99]:
import time
def query(sample_query):
    t = time.time()
    query_vector = model.encode(sample_query)
    print(f"Encode took {time.time()-t}s")
    
    t = time.time()
    scores = cosine_similarity(query_vector, vectors)
    print(f"Search took {time.time()-t}s")
    
    top_scores_ids = np.argsort(scores)
    top_scores_ids = top_scores_ids[::-1][:15]
    for top_id in top_scores_ids:
        tmp = metadata.iloc[idxs[top_id]]
        print("Index:", idxs[top_id])
        print("Title: ", tmp.title_en)
        print("Author: ", tmp.author)
        print("Abstract: ", tmp.abstract_en)
        print("Keywords: ", tmp.keywords_en)
        print("\n##############################################################################\n")

In [90]:
query_vector = model.encode("ddos attack")

In [104]:
query("ddos attack")

Encode took 0.026331424713134766s
Search took 0.0011370182037353516s
Index: 14
Title:  Delayed ettringite formation in fiber cement.
Author:  Davila, Felipe Jaime (Catálogo USP)
Abstract:  The curing temperature using by the fibercement industry as a catalyst in the hydration of cement, hypothesizes expansion caused by delayed ettringite formation. To approach this hypothesis, this study looked at two groups of samples in order to study its chemical and physical behavior individually. One group was composed of cement paste with 35% substitution of limestone filler, and the other by fibercement with addition of PVA and cellulose. In the isothermal calorimetry test was observed the kinetics reaction of both pastes, one cured at 23 °C and the other at 85 °C, the latter showed a catalytic effect in his reactions. XRD and DTG/TG showed the presence of ettringite at 23 °C for all hydration ages exhibiting a trend of increase in the peak intensity and amount, respectively in function of time,

In [205]:
query("ddos attack using machine learning")

Index: 27488
Title:  Machine learning in complex networks
Author:  Breve, Fabricio Aparecido (Catálogo USP)
Abstract:  Complex networks is a recent and active scientific research field, which concerns large scale networks with non-trivial topological structure, such as computer networks, telecommunication networks, transport networks, social networks and biological networks. Many of these networks are naturally divided into communities or modules and, therefore, uncovering their structure is one of the main problems related to complex networks study. This problem is related with the machine learning field, which is concerned with the design and development of algorithms and techniques which allow computers to learn, or increase their performance based on experience. Some of the problems identified in traditional learning techniques include: difficulties in identifying irregular forms in the attributes space; uncovering overlap structures of groups or classes, which occurs when elements

In [240]:
query("ddos attack")

Encode took 0.14992904663085938s
Encode took 275.84169483184814s
Index: 53793
Title:  Method for mitigating against distributed denial of service attacks using multi-agent system.
Author:  Pereira, João Paulo Aragão (Catálogo USP)
Abstract:  The quality of service offered by the Internet Service Provider (ISP) depends directly on the amount of resources available at that time. In recent decades, this quality has been affected by the frequent and intense attacks that consume these resources, such as the Distributed Denial of Service (DDoS) attacks. In order to make the ISPs network more resilient to different types of DDoS attacks, techniques have been developed against such attacks over the past few years. Aiming to contribute to the improvement of such mechanisms, this dissertation presents a reactive autonomous method for detecting and mitigating DDoS attacks using a Multi-Agent system (MAS), in networks of ISPs. The main property of the proposed method is to identify characteristic 

In [241]:
query("ddos attack using machine learning")

Encode took 0.15382623672485352s
Encode took 356.5039610862732s
Index: 97601
Title:  Mitigating DDoS attacks on IoT through machine learning and network functions virtualization
Author:  Oliveira, Guilherme Werneck de (Catálogo USP)
Abstract:  The Internet of Things (IoT) has undergone a rapid popularization, reaching a wide range of application domains. As a consequence, more and more IoT devices with diverse characteristics are deployed in a variety of public and private environments, progressively becoming common objects of everyday life. On the other hand, the physical infrastructure of heterogeneous systems is complex and requires efficient and dynamic solutions for managing network performance and security, at a level that allows standardized deployment and easy replication in smart industries, buildings and cities. An approach that has been gaining ground when the intention is to respond to security threats on IoT is the Network Functions Virtualization (NFV) usage. The literatu